In [1]:
from sklearn.metrics import mean_absolute_error as MSE
from cmcrameri import cm
from tqdm import tqdm
import seaborn as sns
from common import *
from models import *

2021-12-15 23:12:08.626189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 23:12:08.626219: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


# Loading our data

In [2]:
df = pd.read_csv("../data/input_data/MAIN_DATASET.csv")

price = df['NO2_price'].values.reshape(-1,1)
fload = df['NO2_load_forecasted'].values.reshape(-1,1)
fgen = df['NO2_generation_forecast'].values.reshape(-1,1)


price_days = seperate_column_to_days(price)
fload_days = seperate_column_to_days(fload)
fgen_days = seperate_column_to_days(fgen)

# Structuring our data

In [3]:
lookbehind = 7
input_width = lookbehind*24
horizon = 24
no_hours = input_width + horizon
stride = 24
hour_in_days = int(no_hours / stride)

price_dataset = []
fload_dataset = []
fgen_dataset = []

for i in range(len(price_days) - hour_in_days+1):
    price_dataset.append(np.concatenate((price_days[i:i+hour_in_days])))
    fload_dataset.append(np.concatenate((fload_days[i:i+hour_in_days])))
    fgen_dataset.append(np.concatenate((fgen_days[i:i+hour_in_days])))

price_dataset = np.array(price_dataset)
fload_dataset = np.array(fload_dataset)
fgen_dataset = np.array(fgen_dataset)

scaler = MinMaxScaler()
price_dataset = scaler.fit_transform(price_dataset[:,:,0])
price_dataset = price_dataset[..., np.newaxis].astype(np.float32)

fload_dataset = scaler.fit_transform(fload_dataset[:,:,0])
fload_dataset = fload_dataset[..., np.newaxis].astype(np.float32)

fgen_dataset = scaler.fit_transform(fgen_dataset[:,:,0])
fgen_dataset = fgen_dataset[..., np.newaxis].astype(np.float32)

dataset_forecast = np.concatenate((price_dataset, fload_dataset, fgen_dataset), axis=2)

n,m,k = dataset_forecast.shape

print(dataset_forecast.shape)


(2145, 192, 3)


# Train test split

In [4]:
train = int(0.7*n)
valid = int(0.9*n)
x_train, X_train = price_dataset[:train, :input_width], dataset_forecast[:train, :input_width]
x_valid, X_valid = price_dataset[train:valid, :input_width], dataset_forecast[train:valid, :input_width]
x_test, X_test = price_dataset[valid:, :input_width], dataset_forecast[valid:, :input_width]

Y = np.empty((n, input_width, horizon))
for step_ahead in range(1, horizon + 1):
    Y[:,:, step_ahead - 1] = dataset_forecast[:,step_ahead:step_ahead + input_width, 0]

Y_train = Y[:train]
Y_valid = Y[train:valid]
Y_test = Y[valid:]

# Setting up the models

## Common parameters

In [5]:
epochs = 2
loss = "mse" 
optimizer = "adam"

univ_neurons = 128

multi_neurons_first = 128
multi_neurons_second = 64
filters = 64
kernel_size = 3
strides = 3

# Grid search parameters

In [6]:
batch_size = [200, 400, 600, 800]
eta_list = np.logspace(-1, -4, 6)

# Setting up the univariate SimpleRNN

In [7]:
rnn = keras.models.Sequential([
    keras.layers.SimpleRNN(univ_neurons, return_sequences=True, input_shape=[None, 1]),
    keras.layers.SimpleRNN(univ_neurons, return_sequences=True),
    keras.layers.Dense(horizon)
])

rnn.compile(loss=loss, optimizer=optimizer)
rnn.fit(x_train, Y_train, epochs=epochs, batch_size = 100, validation_data=(x_valid, Y_valid))
Y_pred_rnn = rnn.predict(x_test)

2021-12-15 23:12:10.391638: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-15 23:12:10.391744: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-15 23:12:10.391807: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2021-12-15 23:12:10.391852: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2021-12-15 23:12:10.391893: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Epoch 1/2
16/16 [==============================] - 5s 236ms/step - loss: 0.0457 - val_loss: 0.0052
Epoch 2/2
16/16 [==============================] - 2s 151ms/step - loss: 0.0057 - val_loss: 0.0031


In [8]:
print(Y_test.shape)
print(Y_pred_rnn.shape)

print(MSE(Y_test[:,-1], Y_pred_rnn[:,-1]))

(215, 168, 24)
(215, 168, 24)
0.13920488719620555


# Setting up the multivariate ConvGRU

In [9]:
print(f"{X_train.shape=}")
print(f"{Y_train.shape=}")
print(f"{X_valid.shape=}")
print(f"{Y_valid.shape=}")

print(f"{X_train.shape=}")
print(f"{Y_train[:,2::3].shape=}")
print(f"{X_valid.shape=}")
print(f"{Y_valid[:,2::3].shape=}")

X_train.shape=(1501, 168, 3)
Y_train.shape=(1501, 168, 24)
X_valid.shape=(429, 168, 3)
Y_valid.shape=(429, 168, 24)
X_train.shape=(1501, 168, 3)
Y_train[:,2::3].shape=(1501, 56, 24)
X_valid.shape=(429, 168, 3)
Y_valid[:,2::3].shape=(429, 56, 24)


In [10]:
gru = keras.models.Sequential([
    keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding="valid", input_shape=[None, k]),
    keras.layers.GRU(multi_neurons_first, return_sequences=True),
    keras.layers.GRU(multi_neurons_second, return_sequences=True),
    keras.layers.Dense(horizon)
])

gru.compile(loss=loss, optimizer=optimizer)
gru.fit(X_train, Y_train[:,2::3], epochs=epochs, batch_size=100, validation_data=(X_valid, Y_valid[:,2::3]))
Y_pred_gru = gru.predict(X_test)

print(MSE(Y_test[:,-1], Y_pred_gru[:,-1]))

Epoch 1/2
16/16 [==============================] - 5s 136ms/step - loss: 0.0173 - val_loss: 0.0118
Epoch 2/2
16/16 [==============================] - 2s 99ms/step - loss: 0.0057 - val_loss: 0.0062
0.16341531596554118
